In [ ]:
###########################################################
# Célula 1
# Criação das bases em CSV, para facilitar meus estudos equnato estiver offline
#############################################################

#Importação dos módulos desta célula de código
from matminer.datasets.convenience_loaders import load_elastic_tensor
from matminer.featurizers.structure import CoulombMatrix
from matminer.featurizers.conversions import StrToComposition
from matminer.featurizers.composition import ElementProperty
from datetime import datetime
import pandas as pd
import logging

print("Start!")

#marca inicio dete bloco
t1 = datetime.now()

#Criação e configuração do log
log = logging.getLogger('Ex2-2')
log.setLevel(logging.DEBUG)
fh = logging.FileHandler('/home/luis/Downloads/Ex2-2.log')
fh.setLevel(logging.DEBUG)
fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s : %(message)s'))
log.addHandler(fh)

#####################################################################
# Criação dos dados de trino e teste com a base Elastic Tensor
#####################################################################
log.info("Start!")
#obter os dados do elastic tensor e remover colunas não utilizadas
df = load_elastic_tensor()  # loads dataset in a pandas DataFrame object
log.info("Terminada leitura do Elastic Tensor.")
#removendo linhas com mais de 30 átomos (Nsites > 30)
df = df[df.nsites<=30]
log.info("Removidas as linhas com nsites > 30.")
df.to_csv("/home/luis/Downloads/elastic_tensor.csv")
unwanted_columns = ["volume", "nsites", "compliance_tensor", "elastic_tensor", 
                    "elastic_tensor_original", "K_Voigt", "G_Voigt", "K_Reuss", "G_Reuss", 
                    "G_VRH", 'elastic_anisotropy']
df = df.drop(unwanted_columns, axis=1)
log.info("Removidas colunas que não serão utilizadas.")

#Criação do ndarray "Y"
y_K_VRH = df['K_VRH'].values
log.info("Criado o ndarray com os dados para 'Y'")
#Exporta em CSV para facilitar o trabalho quando eu estiver offline.
pd.DataFrame(y_K_VRH).to_csv("/home/luis/Downloads/y_K_VRH.csv") 
log.info("Criado o arquivo y_K_VRH.CSV com os dados para 'Y' para importar nos módulos posteriores.")
log.info("Ínicio dos dados de y_K_VRH:")
log.info(pd.DataFrame(y_K_VRH).head())

###################################################
# Primeiro featurizer: CoulombMatrix
###################################################
#Criação do dataframe "X" com a coulombmatrix
X = df['structure'].as_matrix()
log.info("Criados dados de X como matriz para usar o CoulombMatrix como Flatten")
cm = CoulombMatrix(flatten=True).fit(X)
log.info("Criada a estrutura da CoulombMatrix com dados de X")
X_CM = cm.featurize_dataframe(df.copy(), "structure")
log.info("Criado um DataFrame X_CM, cópia de X, incluindo os dados da CoulombMatrix ")
# exclui demais colunas, para ficar somente os dados da coulombmatrix
excluded = ["K_VRH", "formula", "material_id", 
           "poisson_ratio", "structure", "space_group"]
X_CM = X_CM.drop(excluded, axis=1)
log.info("Excluídas de X_CM as colunas que não serão usadas.")
#Exporta em CSV para facilitar o trabalho quando eu estiver offline.
X_CM.to_csv("/home/luis/Downloads/X_CM.csv") 
log.info("Criado o arquivo X_CM.CSV com os dados de X_CM para importar nos módulos posteriores.")
log.info("Início do dataframe com a CoulombMatrix:")
log.info(X_CM.head())

# Inclui couna "composition" para os próximos featurizers
df = StrToComposition().featurize_dataframe(df, "formula")
log.info("Incluido no Dataframe inicial os dados de Composition (feturizer), para utilização na criação do ElementProperty")

###################################################
# Segundo featurizer: ElementProperty
###################################################
#Criação do dataframe "X" com as informações de EwaldEnergy
ep = ElementProperty.from_preset(preset_name="magpie")
log.info("Criado elemento para featurizer ElementProperty com as informações de EwaldEnergy")
X_EP = ep.featurize_dataframe(df.copy(), col_id="composition")  # Inlui a coluna "composition"
log.info("Criado um DataFrame X_EP, cópia de X, incluindo os dados de ElementProperty")
#exclui demais colunas, para ficar somente os dados de ElementProperty
excluded = ["K_VRH", "formula", "material_id", 
            "poisson_ratio", "structure", "space_group", "composition"]
X_EP = X_EP.drop(excluded, axis=1)
log.info("Excluídas de X_EP as colunas que não serão usadas.")
#Exporta em CSV para facilitar o trabalho quando eu estiver offline.
X_EP.to_csv("/home/luis/Downloads/X_EP.csv") 
log.info("Criado o arquivo X_EP.CSV com os dados de X_EP para importar nos módulos posteriores.")
log.info("Início do dataframe com o ElementProperty")
log.info(X_EP.head())

#maca fim do bloco e tempo gasto
log.info("Finish!")
log.info("Tempo gasto %s" % (datetime.now() - t1))

print("Finish!")




In [1]:
###########################################################
# Célula 2
# Predição com estimator KernelRidge e CoulombMatrix 
#############################################################

# evitando as mensagens dw warning
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
 

#Importação dos módulos desta célula de código
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from matplotlib import pyplot
from datetime import datetime
import pandas as pd
import matplotlib.patches as mpatches
import numpy
import logging

print("Start!")

#marca inicio dete bloco
t1 = datetime.now()

#Criação e configuração do log
log = logging.getLogger('Ex2-2')
log.setLevel(logging.INFO) # ou logging.DEBUG
fh = logging.FileHandler('/home/luis/Downloads/Ex2-2-Celula2.log')
fh.setLevel(logging.INFO) # ou logging.DEBUG
fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s : %(message)s'))
log.addHandler(fh)


# importação dos CSVs para os Dataframes
log.info("Start!")
y_K_VRH = pd.read_csv("/home/luis/Downloads/y_K_VRH.csv")
log.info("Carregado o dataframe com y_K_VRH")
log.info(y_K_VRH.columns)
log.info(y_K_VRH.describe())
X_EP = pd.read_csv("/home/luis/Downloads/X_EP.csv")
log.info("Carregado o dataframe com X_EP")
log.info(X_EP.columns)
log.info(X_EP.describe())
X_CM = pd.read_csv("/home/luis/Downloads/X_CM.csv")
log.info("Carregado o dataframe com y_X_CM")
log.info(X_CM.columns)
log.info(X_CM.describe())


#Separação dos dados de treino e teste
train_X_CM, val_X_CM, train_y_CM, val_y_CM = train_test_split(X_CM, y_K_VRH, random_state=1)
log.info("Criados os dados de train e test para o Dtaframe com CoulombMatrix: X_CM")
log.info(train_X_CM.describe())
log.info(val_X_CM.describe())
train_X_EP, val_X_EP, train_y_EP, val_y_EP = train_test_split(X_EP, y_K_VRH, random_state=1)
log.info("Criados os dados de train e test para o Dtaframe com ElementProperty: X_EP")
log.info(train_X_EP.describe())
log.info(val_X_EP.describe())

#criação de lista com dados de treino e teste
#featurizers = [{"featurizer":["CoulombMatrix"],
#         "train_X": [train_X_CM],
#         "train_y": [train_y_CM],
#         "val_X": [val_X_CM],
#         "val_y": [val_y_CM]},
#        {"featurizer":["ElementProperty"],
#         "train_X": [train_X_EP],
#         "train_y": [train_y_EP],
#         "val_X": [val_X_EP],
#         "val_y": [val_y_EP]}
#       ]




# Lista de Kernels que serão utilizados e respectivas faixas de parâmetros para avaliação
# Os parâmetros utilizados são os disponíveis para os estimators. Este parâmentros podem ser 
# obtidos com a função get_params() de cada estimator
# Para os que vou sar neste estudo, são:
# KerlRidge -> 
# {alpha': 1, 'coef0': 1, 'degree': 3, 'gamma': None, 'kernel': 'linear', 'kernel_params': None}
# SVR ->
# {'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 'auto_deprecated', 
# 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
# Os valores iniciais de cada parâmetros foram definido em um range específico e depois 
# serão ajustados estes ranges conforme os resultados:
# alpha: Fibonacci de 1 a 34
# gamma: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9
# degree: Fibonacci de 1 a 8
# coef0: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9
parameters = [{'kernel': ['linear'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34]}, 
                    {'kernel': ['rbf'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34], 
                     'gamma': [0.1, 0.3, 0.5, 0.7, 0.9]},
                     {'kernel': ['polynomial'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21], 
                     'gamma': [0.1, 0.3, 0.5, 0.7],
                     'degree': [1, 2, 3, 5],
                     'coef0': [0.1, 0.3, 0.5, 0.7]},
                    {'kernel': ['cosine'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34]}, 
                    {'kernel': ['laplacian'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34], 
                     'gamma': [0.1, 0.3, 0.5, 0.7, 0.9]}
                    ]

log.info("Kernels e parâmetros para o GrisSearchCV com KernelRidge:")
log.info(parameters)

# scores para avaliação de kernels lineares
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise
scores = ['r2', 'neg_mean_absolute_error',  'neg_mean_squared_error']
log.info("Scores para o GridSearchCV:")
log.info(scores)

# esta lista de scores pode ser passada direta para o parametro scoring do GridSearchCV, 
#mas foi usada em um laço de "for" para melhor clareza no log de execução
for score in scores: 
    print("-----------------------------------------------------------")
    print()
    print("# Avaliandodados com KernelRidge e score = %s" % score)
    log.info("# Tuning hyper-parameters para KernelRidge e score %s" % score)
    print()

    # percorrendo os featurizers para fazer a predição
    #print("Avaliando o featurizer: %s" % ft["featurizer"])
    #log.info("Avaliando o featurizer: %s" % ft["featurizer"])

    clf = GridSearchCV(KernelRidge(), parameters, cv=2, verbose=1,  
                       scoring='%s' % score, n_jobs=-1)
    log.info("Criado o objeto GridSearchCV:")
    log.info(clf)

    print()
    print("Score com os dados de treino:")
    log.info("Score com os dados de treino:")
    
    #######################################
    # tive que fixar o featurizer pois o grid que criei está perdendo o formato de pandas
    # precisa acertar depois !!!!!!!!!!!
    clf.fit(train_X_CM, train_y_CM)

    log.info("Melhores parâmetros encontrados:")
    log.info(clf.best_params_)
    print("Melhores parâmetros encontrados:")
    print(clf.best_params_)
    
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
        log.info("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

    print()
    print("Score com os dados de teste (avaliação de overfit):")
    log.info("Score com os dados de teste (avaliação de overfit):")
    
    #######################################
    # tive que fixar o featurizer pois o grid que criei está perdendo o formato de pandas
    # precisa acertar depois !!!!!!!!!!!
    clf.fit(val_X_CM, val_y_CM)

    log.info("Melhores parâmetros encontrados:")
    log.info(clf.best_params_)
    print("Melhores parâmetros encontrados:")
    print(clf.best_params_)
    
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
        log.info("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

    

log.info("Finish!")
print("Finish!")


Start!
-----------------------------------------------------------

# Avaliandodados com KernelRidge e score = r2


Score com os dados de treino:
Fitting 2 folds for each of 544 candidates, totalling 1088 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   33.1s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   58.5s
[Parallel(n_jobs=-1)]: Done 1088 out of 1088 | elapsed:  1.3min finished


Melhores parâmetros encontrados:
{'alpha': 2, 'kernel': 'cosine'}
-323.343 (+/-641.832) for {'alpha': 1, 'kernel': 'linear'}
-161.099 (+/-318.004) for {'alpha': 2, 'kernel': 'linear'}
-96.271 (+/-188.893) for {'alpha': 3, 'kernel': 'linear'}
-45.665 (+/-88.512) for {'alpha': 5, 'kernel': 'linear'}
-21.357 (+/-40.728) for {'alpha': 8, 'kernel': 'linear'}
-9.308 (+/-17.421) for {'alpha': 13, 'kernel': 'linear'}
-4.067 (+/-7.529) for {'alpha': 21, 'kernel': 'linear'}
-1.870 (+/-3.496) for {'alpha': 34, 'kernel': 'linear'}
-3.231 (+/-0.025) for {'alpha': 1, 'gamma': 0.1, 'kernel': 'rbf'}
-3.231 (+/-0.025) for {'alpha': 1, 'gamma': 0.3, 'kernel': 'rbf'}
-3.231 (+/-0.025) for {'alpha': 1, 'gamma': 0.5, 'kernel': 'rbf'}
-3.231 (+/-0.025) for {'alpha': 1, 'gamma': 0.7, 'kernel': 'rbf'}
-3.231 (+/-0.025) for {'alpha': 1, 'gamma': 0.9, 'kernel': 'rbf'}
-3.231 (+/-0.025) for {'alpha': 2, 'gamma': 0.1, 'kernel': 'rbf'}
-3.231 (+/-0.025) for {'alpha': 2, 'gamma': 0.3, 'kernel': 'rbf'}
-3.231 (+/-0.

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 1088 out of 1088 | elapsed:   18.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhores parâmetros encontrados:
{'alpha': 2, 'kernel': 'cosine'}
-34.814 (+/-49.407) for {'alpha': 1, 'kernel': 'linear'}
-35.717 (+/-47.440) for {'alpha': 2, 'kernel': 'linear'}
-36.025 (+/-46.692) for {'alpha': 3, 'kernel': 'linear'}
-36.214 (+/-46.077) for {'alpha': 5, 'kernel': 'linear'}
-36.215 (+/-45.752) for {'alpha': 8, 'kernel': 'linear'}
-36.040 (+/-45.597) for {'alpha': 13, 'kernel': 'linear'}
-35.664 (+/-45.588) for {'alpha': 21, 'kernel': 'linear'}
-35.025 (+/-45.698) for {'alpha': 34, 'kernel': 'linear'}
-3.324 (+/-0.355) for {'alpha': 1, 'gamma': 0.1, 'kernel': 'rbf'}
-3.324 (+/-0.355) for {'alpha': 1, 'gamma': 0.3, 'kernel': 'rbf'}
-3.324 (+/-0.355) for {'alpha': 1, 'gamma': 0.5, 'kernel': 'rbf'}
-3.324 (+/-0.355) for {'alpha': 1, 'gamma': 0.7, 'kernel': 'rbf'}
-3.324 (+/-0.355) for {'alpha': 1, 'gamma': 0.9, 'kernel': 'rbf'}
-3.324 (+/-0.355) for {'alpha': 2, 'gamma': 0.1, 'kernel': 'rbf'}
-3.324 (+/-0.355) for {'alpha': 2, 'gamma': 0.3, 'kernel': 'rbf'}
-3.324 (+/-0.

...

[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 316 tasks      | elapsed:   24.0s
[Parallel(n_jobs=-1)]: Done 566 tasks      | elapsed:   47.0s
[Parallel(n_jobs=-1)]: Done 916 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1088 out of 1088 | elapsed:  1.4min finished


Melhores parâmetros encontrados:
{'alpha': 13, 'coef0': 0.7, 'degree': 1, 'gamma': 0.1, 'kernel': 'polynomial'}
-1706649.625 (+/-3374481.550) for {'alpha': 1, 'kernel': 'linear'}
-853219.886 (+/-1671369.988) for {'alpha': 2, 'kernel': 'linear'}
-512186.452 (+/-992389.187) for {'alpha': 3, 'kernel': 'linear'}
-245921.349 (+/-464581.661) for {'alpha': 5, 'kernel': 'linear'}
-117979.628 (+/-213419.411) for {'alpha': 8, 'kernel': 'linear'}
-54520.017 (+/-90994.147) for {'alpha': 13, 'kernel': 'linear'}
-26885.420 (+/-39092.672) for {'alpha': 21, 'kernel': 'linear'}
-15291.604 (+/-17958.703) for {'alpha': 34, 'kernel': 'linear'}
-224756.313 (+/-1021.068) for {'alpha': 1, 'gamma': 0.1, 'kernel': 'rbf'}
-224756.313 (+/-1021.068) for {'alpha': 1, 'gamma': 0.3, 'kernel': 'rbf'}
-224756.313 (+/-1021.068) for {'alpha': 1, 'gamma': 0.5, 'kernel': 'rbf'}
-224756.313 (+/-1021.068) for {'alpha': 1, 'gamma': 0.7, 'kernel': 'rbf'}
-224756.313 (+/-1021.068) for {'alpha': 1, 'gamma': 0.9, 'kernel': 'rbf'

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 744 tasks      | elapsed:   13.2s


Melhores parâmetros encontrados:
{'alpha': 1, 'kernel': 'cosine'}
-171642.152 (+/-217415.308) for {'alpha': 1, 'kernel': 'linear'}
-176734.905 (+/-206476.135) for {'alpha': 2, 'kernel': 'linear'}
-178489.391 (+/-202351.210) for {'alpha': 3, 'kernel': 'linear'}
-179602.780 (+/-199025.770) for {'alpha': 5, 'kernel': 'linear'}
-179677.081 (+/-197370.095) for {'alpha': 8, 'kernel': 'linear'}
-178810.476 (+/-196761.768) for {'alpha': 13, 'kernel': 'linear'}
-176873.423 (+/-197107.007) for {'alpha': 21, 'kernel': 'linear'}
-173552.372 (+/-198331.426) for {'alpha': 34, 'kernel': 'linear'}
-213296.530 (+/-19929.926) for {'alpha': 1, 'gamma': 0.1, 'kernel': 'rbf'}
-213296.530 (+/-19929.926) for {'alpha': 1, 'gamma': 0.3, 'kernel': 'rbf'}
-213296.530 (+/-19929.926) for {'alpha': 1, 'gamma': 0.5, 'kernel': 'rbf'}
-213296.530 (+/-19929.926) for {'alpha': 1, 'gamma': 0.7, 'kernel': 'rbf'}
-213296.530 (+/-19929.926) for {'alpha': 1, 'gamma': 0.9, 'kernel': 'rbf'}
-213296.530 (+/-19929.926) for {'alp

[Parallel(n_jobs=-1)]: Done 1088 out of 1088 | elapsed:   18.2s finished


In [3]:
 ###########################################################
# Célula 3
# Predição com estimator KernelRidge e ElementProperty 
#############################################################

# evitando as mensagens dw warning
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
 

#Importação dos módulos desta célula de código
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from matplotlib import pyplot
from datetime import datetime
import pandas as pd
import matplotlib.patches as mpatches
import numpy
import logging

print("Start!")

#marca inicio dete bloco
t1 = datetime.now()

#Criação e configuração do log
log = logging.getLogger('Ex2-2')
log.setLevel(logging.INFO) # ou logging.DEBUG
fh = logging.FileHandler('/home/luis/Downloads/Ex2-2-Celula2.log')
fh.setLevel(logging.INFO) # ou logging.DEBUG
fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s : %(message)s'))
log.addHandler(fh)


# importação dos CSVs para os Dataframes
log.info("Start!")
y_K_VRH = pd.read_csv("/home/luis/Downloads/y_K_VRH.csv")
log.info("Carregado o dataframe com y_K_VRH")
log.info(y_K_VRH.columns)
log.info(y_K_VRH.describe())
X_EP = pd.read_csv("/home/luis/Downloads/X_EP.csv")
log.info("Carregado o dataframe com X_EP")
log.info(X_EP.columns)
log.info(X_EP.describe())
X_CM = pd.read_csv("/home/luis/Downloads/X_CM.csv")
log.info("Carregado o dataframe com y_X_CM")
log.info(X_CM.columns)
log.info(X_CM.describe())


#Separação dos dados de treino e teste
train_X_CM, val_X_CM, train_y_CM, val_y_CM = train_test_split(X_CM, y_K_VRH, random_state=1)
log.info("Criados os dados de train e test para o Dtaframe com CoulombMatrix: X_CM")
log.info(train_X_CM.describe())
log.info(val_X_CM.describe())
train_X_EP, val_X_EP, train_y_EP, val_y_EP = train_test_split(X_EP, y_K_VRH, random_state=1)
log.info("Criados os dados de train e test para o Dtaframe com ElementProperty: X_EP")
log.info(train_X_EP.describe())
log.info(val_X_EP.describe())

#criação de lista com dados de treino e teste
#featurizers = [{"featurizer":["CoulombMatrix"],
#         "train_X": [train_X_CM],
#         "train_y": [train_y_CM],
#         "val_X": [val_X_CM],
#         "val_y": [val_y_CM]},
#        {"featurizer":["ElementProperty"],
#         "train_X": [train_X_EP],
#         "train_y": [train_y_EP],
#         "val_X": [val_X_EP],
#         "val_y": [val_y_EP]}
#       ]




# Lista de Kernels que serão utilizados e respectivas faixas de parâmetros para avaliação
# Os parâmetros utilizados são os disponíveis para os estimators. Este parâmentros podem ser 
# obtidos com a função get_params() de cada estimator
# Para os que vou sar neste estudo, são:
# KerlRidge -> 
# {alpha': 1, 'coef0': 1, 'degree': 3, 'gamma': None, 'kernel': 'linear', 'kernel_params': None}
# SVR ->
# {'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 'auto_deprecated', 
# 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
# Os valores iniciais de cada parâmetros foram definido em um range específico e depois 
# serão ajustados estes ranges conforme os resultados:
# alpha: Fibonacci de 1 a 34
# gamma: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9
# degree: Fibonacci de 1 a 8
# coef0: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9
parameters = [{'kernel': ['linear'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34]}, 
                    {'kernel': ['rbf'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34], 
                     'gamma': [0.1, 0.3, 0.5, 0.7, 0.9]},
                     {'kernel': ['polynomial'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21], 
                     'gamma': [0.1, 0.3, 0.5, 0.7],
                     'degree': [1, 2, 3, 5],
                     'coef0': [0.1, 0.3, 0.5, 0.7]},
                    {'kernel': ['cosine'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34]}, 
                    {'kernel': ['laplacian'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34], 
                     'gamma': [0.1, 0.3, 0.5, 0.7, 0.9]}
                    ]

log.info("Kernels e parâmetros para o GrisSearchCV com KernelRidge:")
log.info(parameters)

# scores para avaliação de kernels lineares
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise
scores = ['r2', 'neg_mean_absolute_error',  'neg_mean_squared_error']
log.info("Scores para o GridSearchCV:")
log.info(scores)

# esta lista de scores pode ser passada direta para o parametro scoring do GridSearchCV, 
#mas foi usada em um laço de "for" para melhor clareza no log de execução
for score in scores: 
    print("-----------------------------------------------------------")
    print()
    print("# Avaliandodados com KernelRidge e score = %s" % score)
    log.info("# Tuning hyper-parameters para KernelRidge e score %s" % score)
    print()

    # percorrendo os featurizers para fazer a predição
    #print("Avaliando o featurizer: %s" % ft["featurizer"])
    #log.info("Avaliando o featurizer: %s" % ft["featurizer"])

    clf = GridSearchCV(KernelRidge(), parameters, cv=2, verbose=1,  
                       scoring='%s' % score, n_jobs=-1)
    log.info("Criado o objeto GridSearchCV:")
    log.info(clf)

    print()
    print("Score com os dados de treino:")
    log.info("Score com os dados de treino:")
    
    #######################################
    # tive que fixar o featurizer pois o grid que criei está perdendo o formato de pandas
    # precisa acertar depois !!!!!!!!!!!
    clf.fit(train_X_EP, train_y_EP)

    log.info("Melhores parâmetros encontrados:")
    log.info(clf.best_params_)
    print("Melhores parâmetros encontrados:")
    print(clf.best_params_)
    
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
        log.info("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

    print()
    print("Score com os dados de teste (avaliação de overfit):")
    log.info("Score com os dados de teste (avaliação de overfit):")
    
    #######################################
    # tive que fixar o featurizer pois o grid que criei está perdendo o formato de pandas
    # precisa acertar depois !!!!!!!!!!!
    clf.fit(val_X_EP, val_y_EP)

    log.info("Melhores parâmetros encontrados:")
    log.info(clf.best_params_)
    print("Melhores parâmetros encontrados:")
    print(clf.best_params_)
    
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
        log.info("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

    

log.info("Finish!")
print("Finish!")    


Start!
-----------------------------------------------------------

# Avaliandodados com KernelRidge e score = r2


Score com os dados de treino:
Fitting 2 folds for each of 544 candidates, totalling 1088 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 194 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 348 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done 598 tasks      | elapsed:   34.1s
[Parallel(n_jobs=-1)]: Done 948 tasks      | elapsed:   55.0s
[Parallel(n_jobs=-1)]: Done 1088 out of 1088 | elapsed:  1.1min finished


Melhores parâmetros encontrados:
{'alpha': 3, 'kernel': 'linear'}
0.911 (+/-0.020) for {'alpha': 1, 'kernel': 'linear'}
0.912 (+/-0.021) for {'alpha': 2, 'kernel': 'linear'}
0.912 (+/-0.021) for {'alpha': 3, 'kernel': 'linear'}
0.912 (+/-0.021) for {'alpha': 5, 'kernel': 'linear'}
0.911 (+/-0.021) for {'alpha': 8, 'kernel': 'linear'}
0.909 (+/-0.020) for {'alpha': 13, 'kernel': 'linear'}
0.907 (+/-0.019) for {'alpha': 21, 'kernel': 'linear'}
0.904 (+/-0.018) for {'alpha': 34, 'kernel': 'linear'}
-3.228 (+/-0.025) for {'alpha': 1, 'gamma': 0.1, 'kernel': 'rbf'}
-3.231 (+/-0.025) for {'alpha': 1, 'gamma': 0.3, 'kernel': 'rbf'}
-3.231 (+/-0.025) for {'alpha': 1, 'gamma': 0.5, 'kernel': 'rbf'}
-3.231 (+/-0.025) for {'alpha': 1, 'gamma': 0.7, 'kernel': 'rbf'}
-3.231 (+/-0.025) for {'alpha': 1, 'gamma': 0.9, 'kernel': 'rbf'}
-3.229 (+/-0.025) for {'alpha': 2, 'gamma': 0.1, 'kernel': 'rbf'}
-3.231 (+/-0.025) for {'alpha': 2, 'gamma': 0.3, 'kernel': 'rbf'}
-3.231 (+/-0.025) for {'alpha': 2, 'g

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 416 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 1088 out of 1088 | elapsed:   18.5s finished


Melhores parâmetros encontrados:
{'alpha': 21, 'coef0': 0.7, 'degree': 1, 'gamma': 0.1, 'kernel': 'polynomial'}
0.832 (+/-0.067) for {'alpha': 1, 'kernel': 'linear'}
0.859 (+/-0.060) for {'alpha': 2, 'kernel': 'linear'}
0.868 (+/-0.056) for {'alpha': 3, 'kernel': 'linear'}
0.876 (+/-0.050) for {'alpha': 5, 'kernel': 'linear'}
0.879 (+/-0.043) for {'alpha': 8, 'kernel': 'linear'}
0.880 (+/-0.037) for {'alpha': 13, 'kernel': 'linear'}
0.880 (+/-0.030) for {'alpha': 21, 'kernel': 'linear'}
0.882 (+/-0.024) for {'alpha': 34, 'kernel': 'linear'}
-3.318 (+/-0.355) for {'alpha': 1, 'gamma': 0.1, 'kernel': 'rbf'}
-3.324 (+/-0.355) for {'alpha': 1, 'gamma': 0.3, 'kernel': 'rbf'}
-3.324 (+/-0.355) for {'alpha': 1, 'gamma': 0.5, 'kernel': 'rbf'}
-3.324 (+/-0.355) for {'alpha': 1, 'gamma': 0.7, 'kernel': 'rbf'}
-3.324 (+/-0.355) for {'alpha': 1, 'gamma': 0.9, 'kernel': 'rbf'}
-3.320 (+/-0.355) for {'alpha': 2, 'gamma': 0.1, 'kernel': 'rbf'}
-3.324 (+/-0.355) for {'alpha': 2, 'gamma': 0.3, 'kernel'

...

Melhores parâmetros encontrados:
{'alpha': 1, 'coef0': 0.1, 'degree': 1, 'gamma': 0.3, 'kernel': 'polynomial'}
-483.925 (+/-79.142) for {'alpha': 1, 'kernel': 'linear'}
-481.542 (+/-83.035) for {'alpha': 2, 'kernel': 'linear'}
-480.726 (+/-84.269) for {'alpha': 3, 'kernel': 'linear'}
-481.467 (+/-83.870) for {'alpha': 5, 'kernel': 'linear'}
-485.188 (+/-81.178) for {'alpha': 8, 'kernel': 'linear'}
-493.282 (+/-76.396) for {'alpha': 13, 'kernel': 'linear'}
-505.874 (+/-70.571) for {'alpha': 21, 'kernel': 'linear'}
-522.249 (+/-64.408) for {'alpha': 34, 'kernel': 'linear'}
-224583.624 (+/-1021.119) for {'alpha': 1, 'gamma': 0.1, 'kernel': 'rbf'}
-224743.581 (+/-1021.070) for {'alpha': 1, 'gamma': 0.3, 'kernel': 'rbf'}
-224755.035 (+/-1021.068) for {'alpha': 1, 'gamma': 0.5, 'kernel': 'rbf'}
-224756.142 (+/-1021.068) for {'alpha': 1, 'gamma': 0.7, 'kernel': 'rbf'}
-224756.287 (+/-1021.068) for {'alpha': 1, 'gamma': 0.9, 'kernel': 'rbf'}
-224638.589 (+/-1021.090) for {'alpha': 2, 'gamma': 

-224752.140 (+/-1021.068) for {'alpha': 3, 'gamma': 0.9, 'kernel': 'laplacian'}
-224516.721 (+/-1021.038) for {'alpha': 5, 'gamma': 0.1, 'kernel': 'laplacian'}
-224725.095 (+/-1021.071) for {'alpha': 5, 'gamma': 0.3, 'kernel': 'laplacian'}
-224745.106 (+/-1021.069) for {'alpha': 5, 'gamma': 0.5, 'kernel': 'laplacian'}
-224750.856 (+/-1021.068) for {'alpha': 5, 'gamma': 0.7, 'kernel': 'laplacian'}
-224753.524 (+/-1021.068) for {'alpha': 5, 'gamma': 0.9, 'kernel': 'laplacian'}
-224595.197 (+/-1021.055) for {'alpha': 8, 'gamma': 0.1, 'kernel': 'laplacian'}
-224735.356 (+/-1021.069) for {'alpha': 8, 'gamma': 0.3, 'kernel': 'laplacian'}
-224748.804 (+/-1021.068) for {'alpha': 8, 'gamma': 0.5, 'kernel': 'laplacian'}
-224752.665 (+/-1021.068) for {'alpha': 8, 'gamma': 0.7, 'kernel': 'laplacian'}
-224754.451 (+/-1021.068) for {'alpha': 8, 'gamma': 0.9, 'kernel': 'laplacian'}
-224652.101 (+/-1021.062) for {'alpha': 13, 'gamma': 0.1, 'kernel': 'laplacian'}
-224742.774 (+/-1021.068) for {'alpha':

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 212 tasks      | elapsed:    4.0s


Melhores parâmetros encontrados:
{'alpha': 21, 'coef0': 0.7, 'degree': 1, 'gamma': 0.1, 'kernel': 'polynomial'}
-884.023 (+/-509.608) for {'alpha': 1, 'kernel': 'linear'}
-745.721 (+/-447.608) for {'alpha': 2, 'kernel': 'linear'}
-693.804 (+/-414.644) for {'alpha': 3, 'kernel': 'linear'}
-655.264 (+/-376.666) for {'alpha': 5, 'kernel': 'linear'}
-638.726 (+/-343.178) for {'alpha': 8, 'kernel': 'linear'}
-631.184 (+/-308.345) for {'alpha': 13, 'kernel': 'linear'}
-625.678 (+/-273.323) for {'alpha': 21, 'kernel': 'linear'}
-617.751 (+/-238.657) for {'alpha': 34, 'kernel': 'linear'}
-212949.988 (+/-19927.495) for {'alpha': 1, 'gamma': 0.1, 'kernel': 'rbf'}
-213292.674 (+/-19929.899) for {'alpha': 1, 'gamma': 0.3, 'kernel': 'rbf'}
-213296.488 (+/-19929.926) for {'alpha': 1, 'gamma': 0.5, 'kernel': 'rbf'}
-213296.529 (+/-19929.926) for {'alpha': 1, 'gamma': 0.7, 'kernel': 'rbf'}
-213296.530 (+/-19929.926) for {'alpha': 1, 'gamma': 0.9, 'kernel': 'rbf'}
-213063.444 (+/-19928.283) for {'alpha

[Parallel(n_jobs=-1)]: Done 1088 out of 1088 | elapsed:   18.0s finished


In [4]:
###########################################################
# Célula 4
# Predição com estimator SVR e CoulombMatrix 
#############################################################

# evitando as mensagens dw warning
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
 

#Importação dos módulos desta célula de código
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from matplotlib import pyplot
from datetime import datetime
import pandas as pd
import matplotlib.patches as mpatches
import numpy
import logging

print("Start!")

#marca inicio dete bloco
t1 = datetime.now()

#Criação e configuração do log
log = logging.getLogger('Ex2-2')
log.setLevel(logging.INFO) # ou logging.DEBUG
fh = logging.FileHandler('/home/luis/Downloads/Ex2-2-Celula2.log')
fh.setLevel(logging.INFO) # ou logging.DEBUG
fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s : %(message)s'))
log.addHandler(fh)


# importação dos CSVs para os Dataframes
log.info("Start!")
y_K_VRH = pd.read_csv("/home/luis/Downloads/y_K_VRH.csv")
log.info("Carregado o dataframe com y_K_VRH")
log.info(y_K_VRH.columns)
log.info(y_K_VRH.describe())
X_EP = pd.read_csv("/home/luis/Downloads/X_EP.csv")
log.info("Carregado o dataframe com X_EP")
log.info(X_EP.columns)
log.info(X_EP.describe())
X_CM = pd.read_csv("/home/luis/Downloads/X_CM.csv")
log.info("Carregado o dataframe com y_X_CM")
log.info(X_CM.columns)
log.info(X_CM.describe())


#Separação dos dados de treino e teste
train_X_CM, val_X_CM, train_y_CM, val_y_CM = train_test_split(X_CM, y_K_VRH, random_state=1)
log.info("Criados os dados de train e test para o Dtaframe com CoulombMatrix: X_CM")
log.info(train_X_CM.describe())
log.info(val_X_CM.describe())
train_X_EP, val_X_EP, train_y_EP, val_y_EP = train_test_split(X_EP, y_K_VRH, random_state=1)
log.info("Criados os dados de train e test para o Dtaframe com ElementProperty: X_EP")
log.info(train_X_EP.describe())
log.info(val_X_EP.describe())

#criação de lista com dados de treino e teste
#featurizers = [{"featurizer":["CoulombMatrix"],
#         "train_X": [train_X_CM],
#         "train_y": [train_y_CM],
#         "val_X": [val_X_CM],
#         "val_y": [val_y_CM]},
#        {"featurizer":["ElementProperty"],
#         "train_X": [train_X_EP],
#         "train_y": [train_y_EP],
#         "val_X": [val_X_EP],
#         "val_y": [val_y_EP]}
#       ]




# Lista de Kernels que serão utilizados e respectivas faixas de parâmetros para avaliação
# Os parâmetros utilizados são os disponíveis para os estimators. Este parâmentros podem ser 
# obtidos com a função get_params() de cada estimator
# Para os que vou sar neste estudo, são:
# KerlRidge -> 
# {alpha': 1, 'coef0': 1, 'degree': 3, 'gamma': None, 'kernel': 'linear', 'kernel_params': None}
# SVR ->
# {'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 'auto_deprecated', 
# 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
# Os valores iniciais de cada parâmetros foram definido em um range específico e depois 
# serão ajustados estes ranges conforme os resultados:
# alpha: Fibonacci de 1 a 34
# gamma: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9
# degree: Fibonacci de 1 a 8
# coef0: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9
parameters = [{'kernel': ['linear'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34]}, 
                    {'kernel': ['rbf'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34], 
                     'gamma': [0.1, 0.3, 0.5, 0.7, 0.9]},
                     {'kernel': ['polynomial'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21], 
                     'gamma': [0.1, 0.3, 0.5, 0.7],
                     'degree': [1, 2, 3, 5],
                     'coef0': [0.1, 0.3, 0.5, 0.7]},
                    {'kernel': ['cosine'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34]}, 
                    {'kernel': ['laplacian'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34], 
                     'gamma': [0.1, 0.3, 0.5, 0.7, 0.9]}
                    ]

log.info("Kernels e parâmetros para o GrisSearchCV com KernelRidge:")
log.info(parameters)

# scores para avaliação de kernels lineares
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise
scores = ['r2', 'neg_mean_absolute_error',  'neg_mean_squared_error']
log.info("Scores para o GridSearchCV:")
log.info(scores)

# esta lista de scores pode ser passada direta para o parametro scoring do GridSearchCV, 
#mas foi usada em um laço de "for" para melhor clareza no log de execução
for score in scores: 
    print("-----------------------------------------------------------")
    print()
    print("# Avaliandodados com KernelRidge e score = %s" % score)
    log.info("# Tuning hyper-parameters para KernelRidge e score %s" % score)
    print()

    # percorrendo os featurizers para fazer a predição
    #print("Avaliando o featurizer: %s" % ft["featurizer"])
    #log.info("Avaliando o featurizer: %s" % ft["featurizer"])

    clf = GridSearchCV(SVR(), parameters, cv=2, verbose=1,  
                       scoring='%s' % score, n_jobs=-1)
    log.info("Criado o objeto GridSearchCV:")
    log.info(clf)

    print()
    print("Score com os dados de treino:")
    log.info("Score com os dados de treino:")
    
    #######################################
    # tive que fixar o featurizer pois o grid que criei está perdendo o formato de pandas
    # precisa acertar depois !!!!!!!!!!!
    clf.fit(train_X_CM, train_y_CM)

    log.info("Melhores parâmetros encontrados:")
    log.info(clf.best_params_)
    print("Melhores parâmetros encontrados:")
    print(clf.best_params_)
    
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
        log.info("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

    print()
    print("Score com os dados de teste (avaliação de overfit):")
    log.info("Score com os dados de teste (avaliação de overfit):")
    
    #######################################
    # tive que fixar o featurizer pois o grid que criei está perdendo o formato de pandas
    # precisa acertar depois !!!!!!!!!!!
    clf.fit(val_X_CM, val_y_CM)

    log.info("Melhores parâmetros encontrados:")
    log.info(clf.best_params_)
    print("Melhores parâmetros encontrados:")
    print(clf.best_params_)
    
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
        log.info("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

    

log.info("Finish!")
print("Finish!")

Start!
-----------------------------------------------------------

# Avaliandodados com KernelRidge e score = r2


Score com os dados de treino:
Fitting 2 folds for each of 544 candidates, totalling 1088 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


ValueError: Invalid parameter alpha for estimator SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
 ###########################################################
# Célula 5
# Predição com estimator SVR e ElementProperty 
#############################################################

# evitando as mensagens dw warning
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
 

#Importação dos módulos desta célula de código
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from matplotlib import pyplot
from datetime import datetime
import pandas as pd
import matplotlib.patches as mpatches
import numpy
import logging

print("Start!")

#marca inicio dete bloco
t1 = datetime.now()

#Criação e configuração do log
log = logging.getLogger('Ex2-2')
log.setLevel(logging.INFO) # ou logging.DEBUG
fh = logging.FileHandler('/home/luis/Downloads/Ex2-2-Celula2.log')
fh.setLevel(logging.INFO) # ou logging.DEBUG
fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s : %(message)s'))
log.addHandler(fh)


# importação dos CSVs para os Dataframes
log.info("Start!")
y_K_VRH = pd.read_csv("/home/luis/Downloads/y_K_VRH.csv")
log.info("Carregado o dataframe com y_K_VRH")
log.info(y_K_VRH.columns)
log.info(y_K_VRH.describe())
X_EP = pd.read_csv("/home/luis/Downloads/X_EP.csv")
log.info("Carregado o dataframe com X_EP")
log.info(X_EP.columns)
log.info(X_EP.describe())
X_CM = pd.read_csv("/home/luis/Downloads/X_CM.csv")
log.info("Carregado o dataframe com y_X_CM")
log.info(X_CM.columns)
log.info(X_CM.describe())


#Separação dos dados de treino e teste
train_X_CM, val_X_CM, train_y_CM, val_y_CM = train_test_split(X_CM, y_K_VRH, random_state=1)
log.info("Criados os dados de train e test para o Dtaframe com CoulombMatrix: X_CM")
log.info(train_X_CM.describe())
log.info(val_X_CM.describe())
train_X_EP, val_X_EP, train_y_EP, val_y_EP = train_test_split(X_EP, y_K_VRH, random_state=1)
log.info("Criados os dados de train e test para o Dtaframe com ElementProperty: X_EP")
log.info(train_X_EP.describe())
log.info(val_X_EP.describe())

#criação de lista com dados de treino e teste
#featurizers = [{"featurizer":["CoulombMatrix"],
#         "train_X": [train_X_CM],
#         "train_y": [train_y_CM],
#         "val_X": [val_X_CM],
#         "val_y": [val_y_CM]},
#        {"featurizer":["ElementProperty"],
#         "train_X": [train_X_EP],
#         "train_y": [train_y_EP],
#         "val_X": [val_X_EP],
#         "val_y": [val_y_EP]}
#       ]




# Lista de Kernels que serão utilizados e respectivas faixas de parâmetros para avaliação
# Os parâmetros utilizados são os disponíveis para os estimators. Este parâmentros podem ser 
# obtidos com a função get_params() de cada estimator
# Para os que vou sar neste estudo, são:
# KerlRidge -> 
# {alpha': 1, 'coef0': 1, 'degree': 3, 'gamma': None, 'kernel': 'linear', 'kernel_params': None}
# SVR ->
# {'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 'auto_deprecated', 
# 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
# Os valores iniciais de cada parâmetros foram definido em um range específico e depois 
# serão ajustados estes ranges conforme os resultados:
# alpha: Fibonacci de 1 a 34
# gamma: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9
# degree: Fibonacci de 1 a 8
# coef0: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9
parameters = [{'kernel': ['linear'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34]}, 
                    {'kernel': ['rbf'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34], 
                     'gamma': [0.1, 0.3, 0.5, 0.7, 0.9]},
                     {'kernel': ['polynomial'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21], 
                     'gamma': [0.1, 0.3, 0.5, 0.7],
                     'degree': [1, 2, 3, 5],
                     'coef0': [0.1, 0.3, 0.5, 0.7]},
                    {'kernel': ['cosine'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34]}, 
                    {'kernel': ['laplacian'], 
                     'alpha': [1, 2, 3, 5, 8, 13, 21, 34], 
                     'gamma': [0.1, 0.3, 0.5, 0.7, 0.9]}
                    ]

log.info("Kernels e parâmetros para o GrisSearchCV com SVR:")
log.info(parameters)

# scores para avaliação de kernels lineares
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise
scores = ['r2', 'neg_mean_absolute_error',  'neg_mean_squared_error']
log.info("Scores para o GridSearchCV:")
log.info(scores)

# esta lista de scores pode ser passada direta para o parametro scoring do GridSearchCV, 
#mas foi usada em um laço de "for" para melhor clareza no log de execução
for score in scores: 
    print("-----------------------------------------------------------")
    print()
    print("# Avaliandodados com SVR e score = %s" % score)
    log.info("# Avaliandodados com SVR e score = %s" % score)
    print()

    # percorrendo os featurizers para fazer a predição
    #print("Avaliando o featurizer: %s" % ft["featurizer"])
    #log.info("Avaliando o featurizer: %s" % ft["featurizer"])

    clf = GridSearchCV(SVR(), parameters, cv=2, verbose=1,  
                       scoring='%s' % score, n_jobs=-1)
    log.info("Criado o objeto GridSearchCV:")
    log.info(clf)

    print()
    print("Score com os dados de treino:")
    log.info("Score com os dados de treino:")
    
    #######################################
    # tive que fixar o featurizer pois o grid que criei está perdendo o formato de pandas
    # precisa acertar depois !!!!!!!!!!!
    clf.fit(train_X_EP, train_y_EP)

    log.info("Melhores parâmetros encontrados:")
    log.info(clf.best_params_)
    print("Melhores parâmetros encontrados:")
    print(clf.best_params_)
    
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
        log.info("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

    print()
    print("Score com os dados de teste (avaliação de overfit):")
    log.info("Score com os dados de teste (avaliação de overfit):")
    
    #######################################
    # tive que fixar o featurizer pois o grid que criei está perdendo o formato de pandas
    # precisa acertar depois !!!!!!!!!!!
    clf.fit(val_X_EP, val_y_EP)

    log.info("Melhores parâmetros encontrados:")
    log.info(clf.best_params_)
    print("Melhores parâmetros encontrados:")
    print(clf.best_params_)
    
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
        log.info("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

    

log.info("Finish!")
print("Finish!")  